In [97]:
!pip install ipykernel

In [98]:
conda install --name -.venv --update-deps --force-reinstall


In [2]:
import numpy as np
import pandas as pd

## ALCALDIAS Y COLONIAS

In [ ]:
df = pd.read_csv("data/victimasFGJ_acumulado_2024_07.csv")

In [3]:
df = df[["colonia_catalogo", "alcaldia_catalogo", "latitud", "longitud"]].drop_duplicates(["colonia_catalogo", "alcaldia_catalogo"])

In [14]:
df["colonia"] = df["colonia_catalogo"].astype(str).str.upper().apply(lambda x: unidecode(x))
df["alcaldia"] = df["alcaldia_catalogo"].astype(str).str.upper().apply(lambda x: unidecode(x))

In [16]:
aux = df[["colonia", "alcaldia", "latitud", "longitud"]]

In [ ]:
aux.to_csv("data/poligonos.csv")

In [138]:
poligonos = pd.read_csv('data/poligonos.csv', encoding='utf-8')

In [139]:
colonias = list(poligonos.colonia.unique())
alcaldias = list(poligonos.alcaldia.unique())


In [ ]:
!pip install Levenshtein


In [140]:
import Levenshtein

In [141]:
def categoria_mas_parecida(cadena, lista_categorias):
    categoria_parecida = None
    distancia_minima = float('inf')
    
    for categoria in lista_categorias:
        distancia = Levenshtein.distance(cadena, categoria)
        
        if distancia < distancia_minima:
            distancia_minima = distancia
            categoria_parecida = categoria
            
    return categoria_parecida

## CODIGOS POSTALES

In [142]:
codigos = pd.read_csv('data/codigo.csv',  encoding='utf-8')

In [143]:
codigos.columns = codigos.iloc[0,:]

In [144]:
codigos.columns = ['codigo_postal', 'estado', 'alcaldia', 'ciudad', 'tipo', 'colonia', 'clave']

In [145]:
codigos

,codigo_postal,estado,alcaldia,ciudad,tipo,colonia,clave
0,C�digo Postal,Estado,Municipio,Ciudad,Tipo de Asentamiento,Asentamiento,Clave de Oficina
1,1220,Ciudad de M�xico,�lvaro Obreg�n,Ciudad de M�xico,Colonia,La Estrella,1401
2,1230,Ciudad de M�xico,�lvaro Obreg�n,Ciudad de M�xico,Colonia,Campo de Tiro los Gamitos,1401
3,1230,Ciudad de M�xico,�lvaro Obreg�n,Ciudad de M�xico,Colonia,Los Gamitos,1401
4,1230,Ciudad de M�xico,�lvaro Obreg�n,Ciudad de M�xico,Colonia,Tlapechico,1401
...,...,...,...,...,...,...,...
1525,1220,Ciudad de M�xico,�lvaro Obreg�n,Ciudad de M�xico,Colonia,Bonanza,1401
1526,1220,Ciudad de M�xico,�lvaro Obreg�n,Ciudad de M�xico,Colonia,Cuevitas,1401
1527,1220,Ciudad de M�xico,�lvaro Obreg�n,Ciudad de M�xico,Colonia,El Cuernito,1401
1528,1220,Ciudad de M�xico,�lvaro Obreg�n,Ciudad de M�xico,Colonia,M�rtires de Tacubaya,1401


In [146]:
codigos = codigos.iloc[1:,:][["alcaldia", "colonia", 'codigo_postal']]

In [147]:
codigos['colonia'] = codigos['colonia'].str.upper()
codigos['colonia_normalizado'] = codigos['colonia'].apply(lambda x: categoria_mas_parecida(x, colonias)) 

In [148]:
codigos['alcaldia'] = codigos['alcaldia'].str.upper()
codigos['alcaldia_normalizado'] = codigos['alcaldia'].apply(lambda x: categoria_mas_parecida(x, alcaldias)) 

# ANALISIS

In [149]:
codigos

,alcaldia,colonia,codigo_postal,colonia_normalizado,alcaldia_normalizado
1,�LVARO OBREG�N,LA ESTRELLA,1220,LA ESTRELLA,ALVARO OBREGON
2,�LVARO OBREG�N,CAMPO DE TIRO LOS GAMITOS,1230,AMPLIACION LOS OLIVOS,ALVARO OBREGON
3,�LVARO OBREG�N,LOS GAMITOS,1230,LOS GAMITOS,ALVARO OBREGON
4,�LVARO OBREG�N,TLAPECHICO,1230,TLAPECHICO,ALVARO OBREGON
5,�LVARO OBREG�N,EL PIRU SANTA FE,1230,EL PIRU SANTA FE,ALVARO OBREGON
...,...,...,...,...,...
1525,�LVARO OBREG�N,BONANZA,1220,BONANZA,ALVARO OBREGON
1526,�LVARO OBREG�N,CUEVITAS,1220,CUEVITAS,ALVARO OBREGON
1527,�LVARO OBREG�N,EL CUERNITO,1220,EL CUERNITO,ALVARO OBREGON
1528,�LVARO OBREG�N,M�RTIRES DE TACUBAYA,1220,MARTIRES DE TACUBAYA,ALVARO OBREGON


In [154]:
prueba1 = codigos.drop_duplicates(['alcaldia', 'alcaldia_normalizado'])
aux = prueba1.groupby(["alcaldia_normalizado"]).count().reset_index()

alcaldias_final = aux.sort_values(['alcaldia'], ascending=False)[["alcaldia_normalizado"]]

In [159]:
alcaldias_final.to_csv("data/alcaldias_final.csv", index=False)

In [160]:
prueba2 = codigos.drop_duplicates(['colonia', 'colonia_normalizado'])

aux = prueba2.groupby(["colonia_normalizado"]).count().reset_index()

aux.sort_values(['colonia'], ascending=False)

,colonia_normalizado,alcaldia,colonia,codigo_postal,alcaldia_normalizado
230,CENTRO,10,10,10,10
1063,SANTA ANITA,7,7,7,7
763,NEXTITLA,6,6,6,6
1014,SAN JUAN,6,6,6,6
1085,SANTA MARIA TICOMAN,5,5,5,5
...,...,...,...,...,...
418,EX HIPODROMO DE PERALVILLO,1,1,1,1
417,EX HACIENDA SAN JUAN DE DIOS,1,1,1,1
416,EX HACIENDA DE TARANGO,1,1,1,1
414,EX ESCUELA DE TIRO,1,1,1,1


In [161]:
x = aux[aux['colonia'] > 1][["colonia_normalizado", "colonia"]]

In [162]:
x.sort_values(["colonia"], ascending=False)

,colonia_normalizado,colonia
230,CENTRO,10
1063,SANTA ANITA,7
763,NEXTITLA,6
1014,SAN JUAN,6
1137,TLALPAN,5
...,...,...
561,LA CRUZ,2
553,LA CANADA,2
539,JOSE LOPEZ PORTILLO,2
516,IXTLAHUACAN,2


In [163]:
z = x.colonia_normalizado

In [164]:
y = codigos[codigos["colonia_normalizado"].isin(z)].drop_duplicates(["colonia", "colonia_normalizado"])

In [112]:
y[y["colonia_normalizado"] == "TLALPAN"][["colonia", "colonia_normalizado"]]

,colonia,colonia_normalizado
215,TLALPAN,TLALPAN
402,TETELPAN,TLALPAN
1325,ALFALFAR,TLALPAN
1485,TLACOPAC,TLALPAN
1494,TIZAPAN,TLALPAN


In [170]:
colonias1 = poligonos[["colonia"]]
colonias2 = codigos[["colonia_normalizado"]]
colonias2.columns = ["colonia"]
cruce_colonias = colonias1.append(colonias2, ignore_index=True)

In [172]:
cruce_colonias = cruce_colonias.drop_duplicates()

In [174]:
cruce_colonias.to_csv("data/colonias_final.csv", index=False)

# CRUCE DE INFORMACION

In [97]:
poligonos

,colonia,alcaldia,latitud,longitud
0,GUADALUPE INN,ALVARO OBREGON,19.361250,-99.183140
1,VICTORIA DE LAS DEMOCRACIAS,AZCAPOTZALCO,19.471810,-99.164580
2,COPILCO EL BAJO,COYOACAN,19.337970,-99.186110
3,AGRICOLA PANTITLAN,IZTACALCO,19.403270,-99.059830
4,PROGRESISTA,IZTAPALAPA,19.354800,-99.063240
...,...,...,...,...
1830,5TO TRAMO 20 DE NOVIEMBRE,GUSTAVO A. MADERO,19.452338,-99.107923
1831,PURISIMA ATLAZOLPA,IZTACALCO,19.380880,-99.106898
1832,INSURGENTES CUICUILCO,TLALPAN,19.302314,-99.191044
1833,LOMAS DE SANTA FE,MIGUEL HIDALGO,19.388743,-99.240090


In [106]:
poligonos.drop_duplicates(["colonia", 'alcaldia']).shape

(1835, 4)

In [99]:
codigos = codigos[["colonia_normalizado", "alcaldia_normalizado", "codigo_postal"]]
codigos

,colonia_normalizado,alcaldia_normalizado,codigo_postal
1,LA ESTRELLA,ALVARO OBREGON,1220
2,AMPLIACION LOS OLIVOS,ALVARO OBREGON,1230
3,LOS GAMITOS,ALVARO OBREGON,1230
4,TLAPECHICO,ALVARO OBREGON,1230
5,EL PIRU SANTA FE,ALVARO OBREGON,1230
...,...,...,...
1525,BONANZA,ALVARO OBREGON,1220
1526,CUEVITAS,ALVARO OBREGON,1220
1527,EL CUERNITO,ALVARO OBREGON,1220
1528,MARTIRES DE TACUBAYA,ALVARO OBREGON,1220


In [121]:
cruce = poligonos.merge(codigos, how='left', left_on=['colonia', 'alcaldia'], right_on = ['colonia_normalizado', 'alcaldia_normalizado'] )


In [124]:
cruce[cruce["alcaldia"] == "ALVARO OBREGON"]

,colonia,alcaldia,latitud,longitud,colonia_normalizado,alcaldia_normalizado,codigo_postal
0,GUADALUPE INN,ALVARO OBREGON,19.361250,-99.183140,GUADALUPE INN,ALVARO OBREGON,1020
6,TOLTECA,ALVARO OBREGON,19.390000,-99.194720,TOLTECA,ALVARO OBREGON,1150
134,TLAPECHICO,ALVARO OBREGON,19.380290,-99.243290,TLAPECHICO,ALVARO OBREGON,1230
142,SAN ANGEL,ALVARO OBREGON,19.345790,-99.190030,SAN ANGEL,ALVARO OBREGON,1000
155,PROGRESO TIZAPAN,ALVARO OBREGON,19.344250,-99.202470,PROGRESO TIZAPAN,ALVARO OBREGON,1080
...,...,...,...,...,...,...,...
1851,COPILCO EL BAJO,ALVARO OBREGON,19.338591,-99.190318,NaN,NaN,NaN
1854,SAN JOSE INSURGENTES,ALVARO OBREGON,19.363014,-99.189134,NaN,NaN,NaN
1857,BARRIO SANTA CATARINA,ALVARO OBREGON,19.351125,-99.178325,NaN,NaN,NaN
1867,DANIEL GARZA,ALVARO OBREGON,19.402033,-99.200501,NaN,NaN,NaN


# ANALISIS 2

### ALCALDIAS

In [136]:
poligonos[["alcaldia"]].drop_duplicates(["alcaldia"])

,alcaldia
0,ALVARO OBREGON
1,AZCAPOTZALCO
2,COYOACAN
3,IZTACALCO
4,IZTAPALAPA
9,VENUSTIANO CARRANZA
10,GUSTAVO A. MADERO
11,XOCHIMILCO
12,MIGUEL HIDALGO
13,CUAUHTEMOC


In [137]:
codigos[["alcaldia_normalizado"]].drop_duplicates(["alcaldia_normalizado"])

,alcaldia_normalizado
1,ALVARO OBREGON
56,LA MAGDALENA CONTRERAS
83,MIGUEL HIDALGO
119,MILPA ALTA
154,TLAHUAC
161,VENUSTIANO CARRANZA
214,TLALPAN
331,XOCHIMILCO
437,COYOACAN
473,CUAJIMALPA DE MORELOS
